In [1]:
import tensorflow as tf
import pathlib
import os
import time

In [2]:
# Setup path
originalDatadir = pathlib.Path("/Volumes/T7/coco/val2017")
destinationDatadir = "/Volumes/T7/coco/recolorize/color"
destinationDatadir_bw = "/Volumes/T7/coco/recolorize/bw"

In [3]:
# List image in directory
def list_image(path: pathlib.Path):
    imglist = path.glob("*.jpg")
    return [str(img_path) for img_path in imglist]


In [4]:
# a bit dirty but it works
def resize_img(path: str):
    file = tf.io.read_file(path)
    image = tf.image.decode_jpeg(file, channels=3)
    shape = tf.shape(image)
    w, h = shape[0], shape[1]
    img = tf.image.resize_with_crop_or_pad(image=image, target_height=256, target_width=256)
    bwimg = tf.image.rgb_to_grayscale(img)
    resized_jpg = tf.io.encode_jpeg(img, optimize_size=True)
    resizedbw_jpg = tf.io.encode_jpeg(bwimg, optimize_size=True)
    return resized_jpg, resizedbw_jpg

In [5]:
start = time.time()
for img in list_image(originalDatadir):
    # get filename
    colorpath = os.path.join(destinationDatadir, os.path.basename(img))
    bwpath = os.path.join(destinationDatadir_bw, os.path.basename(img))

    # resize and convert to bw
    resized, resizedbw = resize_img(img)

    # save
    tf.io.write_file(bwpath, resizedbw)
    tf.io.write_file(colorpath, resized)

print("Elapsed : {}s".format(time.time() - start))


2021-12-14 21:06:37.354698: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-14 21:06:37.354791: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1
Elapsed : 60.558111906051636s
